In [ ]:
# TẢI DỮ LIỆU VÀ CÁC PACKAGE CẦN THIẾT  

import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns 
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

file_path = "../data/input_data.xlsx"
xls = pd.ExcelFile(file_path)
original_df = xls.parse(sheet_name="Sheet1")

# display(original_df.head(5))
display(original_df.shape) # data có 1250 dòng và 84 cột 
# display(original_df.columns) # có 84 cột, một số tên không thuận tiện cho thao tác, và một số cột sẽ không cần thiết 

(1250, 84)

In [2]:
# ĐỔI TIÊN CÁC CỘT 

columns_mapper = {
    "Number": "number",
    "BMI": "bmi",
    "Insulin": "insulin",
    "Glucose_AC": "glucose_ac",
    "Cortisol": "cortisol",
    "hs-CRP": "hs_CRP",
    "Waist": "waist",
    "Hip": "hip",
    "Date_follow": "date_follow",
    "Waist_hip_ratio": "waist_hip_ratio",
    "Height": "height",
    "Weight": "weight",
    "Birth": "birth",
    "family income": "family_income",
    "smoking during pregnancy": "pregnancy_smoking",
    "smoking frequency per day": "smoking_frequency_per_day",
    "Gestational weight gain": "gestational_weight_gain",
    "Preterm birth": "preterm_birth",
    "Gestational age at delivery(week)": "gestational_age_week",
    "Birth weight(gram)": "birth_weight_gram",
    "Exclusive breastfeeding time(month)": "exclusive_breastfeeding_month",
    "Mixed breastfeeding time(month)": "mixed_breastfeeding_month",
    "Father_diabetes": "father_diabetes",
    "Mother_diabetes": "mother_diabetes",
    "Sedentary lifestyle(hours/per day)": "sedentary_lifestyle_hour_day",
    "low physical activity(hours/per day)": "low_physical_activity_hour_day",
    "snoring(times/week)": "snoring_times_week",
    "Sleep apnea(times/week)": "sleep_apnea_times_week",
    "CES-D": "CES_D",
}

df_columns_renamed = original_df.rename(columns=columns_mapper, inplace=False)

# display(df_columns_renamed.head(5))
# display(df_columns_renamed.shape)
# display(df_columns_renamed.columns)

In [3]:
# LOẠI BỎ CÁC DÒNG LẶP LẠI TOÀN BỘ 

df_no_duplicates = df_columns_renamed.drop_duplicates().copy().reset_index(drop=True)
# vì bảng dữ liệu sau thao tác này sẽ còn tiếp tục bị biến đổi nên dùng lệnh copy()
# nhớ reset_index, drop = True giúp loại bỏ cột index cũ để tạo index mới

# display(df_columns_no_duplicates.shape) # Chỉ có 2 dòng lặp lại hoàn toàn 

In [4]:
# NHẬN ĐỊNH VỀ BẢNG DỮ LIỆU 

df_no_duplicates.info()

# điều ta quan tâm là những biến số nào có missing data và kiểu dữ liệu của các biến số
# các biến số không có missing data: number, hospital, blood_date. Tất cả các biến số còn lại có missing data 
# có các định dạng dữ liệu là: object, float64, datetime64[ns]     
# có biến số nào mà định dạng dữ liệu không phù hợp không? 
# kì vọng là số nhưng hiển thị là chữ
# - number
# - glucose_ac 
# - cortisol
# - hs_CRP

# kì vọng là chữ nhưng hiển thị là số 
# - sex
# - pregnancy_smoking
# - GDM
# - preterm_birth
# - father_diabetes
# - mother_diabetes
# - education_level

# dường như là dữ liệu dạng boolean
# - sex
# - pregnancy_smoking
# - GDM
# - preterm_birth
# - father_diabetes
# - mother_diabetes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 84 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   number                          1246 non-null   object        
 1   hospital                        1246 non-null   object        
 2   age                             1244 non-null   float64       
 3   zbmi                            958 non-null    float64       
 4   bmi                             961 non-null    float64       
 5   blood_date                      1246 non-null   datetime64[ns]
 6   insulin                         418 non-null    float64       
 7   glucose_ac                      625 non-null    object        
 8   cholesterol                     483 non-null    float64       
 9   TG                              471 non-null    float64       
 10  LDL                             16 non-null     float64       
 11  HDL 

In [5]:
# KIỂM TRA TÍNH TƯƠNG THÍCH KIỂU DỮ LIỆU CỦA BIẾN SỐ KÌ VỌNG LÀ SỐ NHƯNG THỰC RA LÀ CHỮ

invalid_object_column = ["glucose_ac", "cortisol", "hs_CRP", "number"]

# for column in invalid_object_column: 
#     display(df_no_duplicates[column].value_counts())

# cột glucose_ac có giá trị 119/121 làm sai lệch dữ liệu => chọn một giá trị là 121
# cột cortisol có giá trị <0.24, <0.054 làm sai lệch dữ liệu => viết hàm thay giá trị "<" thành nan
# cột hs_CRP có giá trị < ... làm sai lệch dữ liệu, khá là nhiều => viết hàm thay giá trị "<" thành nan
# cột number có một số giá trị vừa chứa chữ, vừa chứa số => chuyển thành str một lần nữa


df_no_duplicates.loc[df_no_duplicates["glucose_ac"] == "119/121", "glucose_ac"] = 121
df_no_duplicates["glucose_ac"] = df_no_duplicates["glucose_ac"].astype("float64")
# df_no_duplicates["glucose_ac"].dtype

mask_1 = df_no_duplicates["cortisol"].astype("str").str.contains("<", na=False)
mask_2 = df_no_duplicates["hs_CRP"].astype("str").str.contains("<", na=False)

df_no_duplicates.loc[mask_1, "cortisol"] = np.nan
df_no_duplicates.loc[mask_2, "hs_CRP"] = np.nan

df_no_duplicates["cortisol"] = df_no_duplicates["cortisol"].astype("float64")
df_no_duplicates["hs_CRP"] = df_no_duplicates["hs_CRP"].astype("float64")
df_no_duplicates["number"] = df_no_duplicates["number"].astype("str")


In [6]:
# KIỂM TRA TÍNH TƯƠNG THÍCH KIỂU DỮ LIỆU CỦA BIẾN SỐ CÓ THỂ LÀ BOOL/CAT 

maybe_bool_columns = ["sex", "pregnancy_smoking", "GDM", "preterm_birth", "father_diabetes", "mother_diabetes"]

# for column in maybe_bool_columns: 
#     display(df_no_duplicates[column].value_counts())

df_no_duplicates["sex"] = df_no_duplicates["sex"].map({1.0: "male", 2.0: "female"}).astype("category")
df_no_duplicates["pregnancy_smoking"] = df_no_duplicates["pregnancy_smoking"].map({0.0: "no", 1.0: "yes"}).astype("category")
df_no_duplicates["GDM"] = df_no_duplicates["GDM"].map({0.0: "no-dia", 1.0: "gmd", 2.0: "other-dia"}).astype("category")
df_no_duplicates["preterm_birth"] = df_no_duplicates["preterm_birth"].map({0.0: "no", 1.0: "yes"}).astype("category")
df_no_duplicates["father_diabetes"] = df_no_duplicates["father_diabetes"].map({0.0: "no", 1.0: "yes"}).astype("category")
df_no_duplicates["mother_diabetes"] = df_no_duplicates["mother_diabetes"].map({0.0: "no", 1.0: "yes"}).astype("category")
df_no_duplicates["education_level"] = df_no_duplicates["education_level"].map({
    1.0: "junior high", 
    2.0: "senior high", 
    3.0: "vocational", 
    4.0: "college", 
    5.0: "graduate"
})
df_no_duplicates["education_level"] = pd.Categorical(df_no_duplicates["education_level"], categories=["junior high", "senior high", "vocational", "college", "graduate"], ordered=True)


# display(df_no_duplicates["sex"].value_counts(dropna=False))
# display(df_no_duplicates["sex"].dtype)
# display(df_no_duplicates["pregnancy_smoking"].value_counts(dropna=False))
# display(df_no_duplicates["pregnancy_smoking"].dtype)
# display(df_no_duplicates["GDM"].value_counts(dropna=False))
# display(df_no_duplicates["GDM"].dtype)
# display(df_no_duplicates["preterm_birth"].value_counts(dropna=False))
# display(df_no_duplicates["preterm_birth"].dtype)
# display(df_no_duplicates["father_diabetes"].value_counts(dropna=False))
# display(df_no_duplicates["father_diabetes"].dtype)
# display(df_no_duplicates["mother_diabetes"].value_counts(dropna=False))
# display(df_no_duplicates["mother_diabetes"].dtype)
# display(df_no_duplicates["education_level"].value_counts(dropna=False))
# display(df_no_duplicates["education_level"].dtype)

In [7]:
# XÁC ĐỊNH LẠI KIỂU DỮ LIỆU CỦA CÁC BIẾN SỐ

object_columns = df_no_duplicates.select_dtypes(include='object').columns.tolist()
display(object_columns)
# có 2 biến số định tính là: ['number', 'hospital']
# hãy cẩn thận với các biến số định tính nhưng là số, nó có thể là số hoặc chữ nhưng được phiên giải là object
# trong cột number vừa có chữ vừa có số, hiển thị là chữ nhưng số thì vẫn là số ???

float_columns = df_no_duplicates.select_dtypes(include='float64').columns.tolist()
display(len(float_columns))
# có 71 biến số định lượng

date_time_columns = df_no_duplicates.select_dtypes(include="datetime64[ns]").columns.tolist()
display(date_time_columns)
# có 4 biến số date_time: ['blood_date', 'date_follow', 'body_date', 'birth']

categorical_columns = df_no_duplicates.select_dtypes(include='category').columns.tolist()
display(categorical_columns)
# có 7 biến số categorical: ['sex', 'pregnancy_smoking', 'GDM', 'preterm_birth', 'father_diabetes', 'mother_diabetes', 'education_level']

['number', 'hospital']

71

['blood_date', 'date_follow', 'body_date', 'birth']

['sex',
 'pregnancy_smoking',
 'GDM',
 'preterm_birth',
 'father_diabetes',
 'mother_diabetes',
 'education_level']

In [8]:
# XÁC ĐỊNH GIÁ TRỊ LẶP LẠI Ở CỘT NUMBER

duplicated_numbers = df_no_duplicates['number'][df_no_duplicates['number'].duplicated(keep=False)]

# display(len(duplicated_numbers)) # có 436 hàng lặp lại ở cột number
display(len(duplicated_numbers.unique())) # có 160 giá trị bị lặp lại
# display(duplicated_numbers.unique()) # hiển thị các giá trị bị lặp lại 
# display(duplicated_numbers.value_counts()) # xem số các giá tị bị lặp lại từ nhiều nhất đến ít nhất

non_duplicated_numbers = df_no_duplicates['number'][~df_no_duplicates['number'].duplicated(keep=False)]
# display(len(non_duplicated_numbers)) # có 810 đối tượng không bị lặp lại
display(len(non_duplicated_numbers.unique())) # có 810 đối tượng không bị lặp lại
# display(non_duplicated_numbers.unique()) 
# display(non_duplicated_numbers.value_counts())

# TỔNG SỐ ĐỐI TƯỢNG QUAN SÁT KHÔNG TÍNH LẶP LẠI LÀ: 160 + 810 = 970

160

810

In [9]:
# THỬ XEM VÀI ĐỐI TƯỢNG BỊ LẶP LẠI TRÔNG NHƯ THẾ NÀO

p_c0458 = df_no_duplicates[df_no_duplicates["number"] == "c0458"]
p_580 = df_no_duplicates[df_no_duplicates["number"] == "580"]
p_c1075 = df_no_duplicates[df_no_duplicates["number"] == "c1075"]

# display(p_c0458)
# display(p_580)
# display(p_c1075)

# Ta thấy gì? 


In [10]:
# TẠO NHÓM THEO CỘT NUMBER

df_miss_insulin = pd.DataFrame(df_no_duplicates.groupby("number")["insulin"].count())
display(df_miss_insulin.value_counts(dropna=False)) 
# có 599 đối tượng không được ghi nhận bất kỳ giá trị insulin nào

df_miss_glucose = pd.DataFrame(df_no_duplicates.groupby("number")["glucose_ac"].count())
display(df_miss_glucose.value_counts(dropna=False)) 
# có 489 đối tượng không được ghi nhận bất kỳ giá trị glucose nào



insulin
0          599
1          333
2           32
3            3
4            3
Name: count, dtype: int64

glucose_ac
0             489
1             406
2              46
3              14
5               7
4               5
6               2
18              1
Name: count, dtype: int64

In [11]:
# XỬ LÝ CÁC GIÁ TRỊ THIẾU Ở CỘT INSULIN

# loại bỏ các dòng thiếu cả insulin
df_filtered_1 = df_no_duplicates[~(df_no_duplicates["insulin"].isna() & df_no_duplicates["glucose_ac"].isna())].copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_filtered_1.to_excel("df_filtered_1.xlsx", index=False)
# df_filtered_1 chứa các hàng không bị thiếu ở cả cột insulin và glucose

# thực hiện phép đếm để có được number của các đối tượng không được ghi nhận bất kì giá trị insulin nào
df_insulin_count = pd.DataFrame(df_filtered_1.groupby("number")["insulin"].count().sort_values())
no_insulin_obs = df_insulin_count[df_insulin_count["insulin"] == 0].index.to_list()

# loại bỏ các đối tượng không được ghi nhận bất kì giá trị insulin nào
df_filtered_2 = df_filtered_1[~df_filtered_1['number'].isin(no_insulin_obs)].reset_index(drop=True).copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_filtered_2.to_excel("df_filtered_2.xlsx", index=False)
# df_filtered_2 chứa các hàng có ít nhất một lần ghi nhận insulin



In [12]:
# XỬ LÝ CÁC GIÁ TRỊ THIẾU Ở CỘT GLUCOSE

# thực hiện phép đếm để có được number của các đối tượng không được ghi nhận bất kì giá trị insulin nào
df_glucose_count = pd.DataFrame(df_filtered_2.groupby("number")["glucose_ac"].count().sort_values())
no_glucose_obs = df_glucose_count[df_glucose_count["glucose_ac"] == 0].index.to_list()

# loại bỏ các đối tượng không được ghi nhận bất kì giá trị insulin nào
df_filtered_3 = df_filtered_2[~df_filtered_2['number'].isin(no_glucose_obs)].copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_filtered_3.to_excel("df_filtered_3.xlsx", index=False)
# df_filtered_3 chứa các hàng có ít nhất một lần ghi nhận glucose và một lần ghi nhận insulin, vẫn còn chứa các hàng lặp lại

In [13]:
# TẠO BẢNG CON CHỨA CÁC HÀNG XUẤT HIỆN 1 LẦN, CÓ ĐẦY ĐỦ INSULIN VÀ GLUCOSE_AC

# tạo bảng chứa những hàng chỉ xuất hiện 1 lần, có đầy đủ insulin và glucose_ac
df_no_dup = df_filtered_3[~df_filtered_3["number"].duplicated(keep=False)].copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_no_dup.to_excel("df_no_dup.xlsx", index=False)
# df_no_dup chứa những hàng chỉ xuất hiện 1 lần, có đầy đủ insulin và glucose_ac

In [14]:
# TẠO BẢNG CON CHỨA CÁC HÀNG BỊ LẶP LẠI, CÓ ĐẦY ĐỦ INSULIN VÀ GLUCOSE_AC

df_dup = df_filtered_3[df_filtered_3["number"].duplicated(keep=False)].copy().reset_index(drop=True)

df_dup_in_glu = df_dup[df_dup["insulin"].notna() & df_dup["glucose_ac"].notna()].copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_dup_in_glu.to_excel("df_dup_in_glu.xlsx", index=False)
# df_dup_in_glu chứa các hàng có đủ cả insulin và glucose, có thể lặp lại hoặc không

df_dup_miss_in_or_glu = df_dup[df_dup["insulin"].isna() | df_dup["glucose_ac"].isna()].copy().reset_index(drop=True)

# xuất ra bảng dữ liệu để kiểm tra
# df_dup_miss_in_or_glu.to_excel("df_dup_miss_in_or_glu.xlsx", index=False)
# df_dup_miss_in_or_glu chứa các hàng bị lặp lại hoặc không, có insulin hoặc glucose bị thiếu

